# Effect of Covid19 based on Region

Reading in saved dataframe from and earlier exercise to get population size and age groups

In [1]:
import pandas as pd
import plotly_express as px

file_path = "Data/Vaccinerade_kommun_och_ålder.xlsx"

df_befolkning = pd.read_excel(file_path)

df_befolkning.head(2)

,Region,Kommun,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal minst 3 doser,Antal minst 4 doser
0,Stockholm,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN
1,Stockholm,Upplands Väsby,16-17,1203,839,755,NaN,NaN


Reading "totalt antal per region" to get cases by region

In [2]:
file_path = "Data/Folkhalsomyndigheten_Covid19.xlsx"
sheet_name = "Totalt antal per region"

df_antal_fall = pd.read_excel(file_path, sheet_name = sheet_name, usecols = "A,B,E,D")

df_antal_fall.head()

,Region,Totalt_antal_fall,Totalt_antal_intensivvårdade,Totalt_antal_avlidna
0,Blekinge,30829,85,184
1,Dalarna,75091,260,544
2,Gotland,11874,46,82
3,Gävleborg,74803,332,754
4,Halland,108822,229,518


In [3]:
# renaming different names of same region to match
df_antal_fall.loc[(df_antal_fall["Region"] == "Sörmland"), "Region"] = "Södermanland"
df_antal_fall.loc[(df_antal_fall["Region"] == "Jämtland Härjedalen"), "Region"] = "Jämtland"

df_antal_fall.loc[(df_antal_fall["Region"] == "Södermanland") | (df_antal_fall["Region"] == "Jämtland")]

,Region,Totalt_antal_fall,Totalt_antal_intensivvårdade,Totalt_antal_avlidna
5,Jämtland,34347,95,197
12,Södermanland,67918,446,662


In [4]:
# setting region to index for easier calculations, since both dataframes share indices, and all values of columns are numerical
df_antal_fall.set_index("Region", inplace = True)
df_antal_fall.head(2)

,Totalt_antal_fall,Totalt_antal_intensivvårdade,Totalt_antal_avlidna
Region,,,
Blekinge,30829,85,184
Dalarna,75091,260,544


---
# KPI: Infection Rate per Region
The amount of a regions population that has been infected with Covid19  
This KPI is relevant for seeing how badly different regions have been effected  
It can also point towards efficiency of infection prevention strategies

In [5]:
data = df_antal_fall["Totalt_antal_fall"].div(df_befolkning.groupby("Region")["Befolkning"].sum(), axis="rows")

fig = px.bar(
    data.sort_values(),
    title="Andel Invånare med Covid19 per Region",
    labels={"value": "Andel av Invånare"},
)

fig.update_layout(showlegend=False)  # hiding legend since it is not needed here

fig.write_html("Visualiseringar/E3_KPI_Infection_Rate.html")
fig.show()

We can also get amount of population who received intensive care, amount of population deceased  
However, these numbers do not take into account the amount of population that was infected, and therefore does not tell the whole story  
In order to get a more accurate representation of this, it is better to calculate the relative amount based off the amount of infected individuals 

---
# KPI: Amount of Intensive Care Patients per Case by Region
The amount of a regions infected population that has received intensive care

This KPI can be useful towards determining the effectiveness of healthcare system in the region,   
especially if combined with data from intensive care units it can be used to determine pressure put on the healthcare system in the region and how well it was handled

Data such as overtime worked by staff, age of patients, and amount of patients requiring intensive care vs patiens who received it would be relevant here

In [6]:
data = df_antal_fall["Totalt_antal_intensivvårdade"].div(df_antal_fall["Totalt_antal_fall"], axis="rows")

fig = px.bar(
    data.sort_values(),
    title="Andel Invånare som Intensivvårdats per Region",
    labels={"value": "Andel av Invånare"},
)

fig.update_layout(showlegend=False)  # hiding legend since it is not needed here

fig.write_html("Visualiseringar/E3_KPI_Intensive_Care_Rate.html")
fig.show()


---
# KPI: Mortality Rate per Region
The amount of a regions infected population that died

Since this takes the amount of infected patients into account, it effectively shows the mortality rate per region  
This can be useful in determining how well the regions healthcare system handled patients  

Important to take into consideration here is my KPI showing effect of virus on different age groups showing that the virus affects older people worse     
If a certain region has an overall higher population age they can expect to see a higher mortality rate    
Likewise areas with lower population age, such as large cities, should expect to see lower mortality rates  

It would also be relevant to see data from the regions healthcare system  
If they were understaffed and overworked, there might be higher mortality rates due to shortcomings in staffing  
Otherwise it might point towards shortcomings in priority of patients, or other difficulties in receiving the correct care  

In [7]:
data = df_antal_fall["Totalt_antal_avlidna"].div(df_antal_fall["Totalt_antal_fall"], axis="rows")

fig = px.bar(
    data.sort_values(),
    title="Andel Invånare som Avlidit per Region",
    labels={"value": "Andel av Invånare"},
)

fig.update_layout(showlegend=False)  # hiding legend since it is not needed here

fig.write_html("Visualiseringar/E3_KPI_Mortality_Rate_Region.html")
fig.show()